In [1]:
import pandas as pd
import numpy as np
import sys
import random
import os
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

2023-07-29 10:31:19.058221: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 10:31:19.243853: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-29 10:31:19.282697: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-29 10:31:19.282723: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

#### The data

In [3]:
kampala_devices = pd.read_csv('../data/kampala_devices.csv', usecols=['lat', 'long', 'id'])
kampala_devices.head()

,id,lat,long
0,930434,0.360209,32.610756
1,718028,0.307500,32.620600
2,912224,0.346460,32.703280
3,930426,0.365500,32.646800
4,930427,0.268900,32.588000


In [4]:
kampala_df = pd.read_csv('../data/kampala_data.csv', parse_dates=['timestamp'])
kampala_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
1,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
2,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
3,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
4,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26


In [5]:
final_df = pd.DataFrame()
# device_ids = []
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value', 'device_number']
# for i, device_id in enumerate(device_ids):
for i, device_id in kampala_devices.id.iteritems():
    device_df = utils.get_device_data(kampala_df, device_id, cols)
#     if len(device_df)!=0:
#         device_ids.append(device_id)
    processed_df = utils.preprocessing(device_df)
#     print(f'{device_id}: {len(processed_df)}')
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5,device_number
0,453031.0,0.356989,32.613888,10.5477,930434
1,453032.0,0.356989,32.613888,16.4250,930434
2,453033.0,0.356989,32.613888,17.7239,930434
3,453034.0,0.356989,32.613888,16.1533,930434
4,453035.0,0.356989,32.613888,18.0123,930434


In [6]:
latitudes = final_df['latitude'].unique()
longitudes = final_df['longitude'].unique()
device_ids = final_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(35, 35, 34)

#### Model training and validation

In [7]:
def lstm(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
#     model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.LSTM(50),
#     tf.keras.layers.Dense(1)
# ])
    
    input_layer = Input(shape=(X_train.shape[1],))
    reshaped_input = tf.expand_dims(input_layer, axis=1)
    
    lstm_layer1 = LSTM(units=50, return_sequences=True)(reshaped_input)
#     dropout_layer1 = Dropout(dropout)(lstm_layer1)

    lstm_layer2 = LSTM(units=50)(lstm_layer1)
#     dropout_layer2 = Dropout(dropout)(lstm_layer2)
    
    output_layer = Dense(units=1)(lstm_layer2)

    model = Model(inputs=input_layer, outputs=output_layer) 

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_split=0.2)
    
#     model = keras.Sequential()
#     model.add(Input(shape=(X_train.shape[1],), name='Input-Layer')) 
#     model.add(LSTM(50, return_sequences=True, input_shape=(window_size, x_train.shape[-1])))
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM((window_size * 2), return_sequences=True))) 
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM(window_size, return_sequences=False))) 
#     model.add(Dense(units=1))
    return model

In [8]:
# def scale_data(X):
#     scaler = MinMaxScaler()
#     X_scaled = X.copy()
#     X_scaled[:, 0] = scaler.fit_transform(X[:, 0].reshape(-1, 1)).flatten()
#     return X_scaled

In [9]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = kampala_df[kampala_df.device_number == device_ids[idx]]
#     assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
#     assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
#     assert(len(train_df.longitude.unique()) == len(longitudes)-1)
#     assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
#     X_train_scaled = scale_data(X_train)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
#     X_test_scaled = scale_data(X_test)
    
    model = lstm(X_train, y_train) #, dropout=0.1)
    y_pred = model.predict(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

In [10]:
len(latitudes)

35

In [11]:
rmse_list, mape_list = [], []
# for i in range(len(latitudes[30:])):
lat_list = list(latitudes)
for lat in latitudes[:4]:
    i = lat_list.index(lat)
#     print(i)
    try:
        rmse, mape = cross_validation(final_df, i)
        rmse_list.append(rmse)
        mape_list.append(mape)
        print(f'Location {i} successful')
    except Exception as e:
        print(e)
        print(f'Location {i} failed')
#         break

mean_rmse = np.mean(rmse_list)
mean_mape = np.mean(mape_list)
mean_rmse, mean_mape

2023-07-29 10:31:29.894084: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-29 10:31:29.894113: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: PL1207-PRO.paris.inria.fr
2023-07-29 10:31:29.894116: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: PL1207-PRO.paris.inria.fr
2023-07-29 10:31:29.894211: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.125.6
2023-07-29 10:31:29.894231: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  525.125.06  Release Build  (dvs-builder@U16-A23-14-1)  Tue May 30 05:12:48 UTC 2023
GCC version:  gcc ve

Epoch 1/1000
1155/1155 [==============================] - 5s 3ms/step - loss: 679029112832.0000 - val_loss: 758114287616.0000
Epoch 2/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 678953287680.0000 - val_loss: 758036234240.0000
Epoch 3/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 678879297536.0000 - val_loss: 757957263360.0000
Epoch 4/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 678805241856.0000 - val_loss: 757878226944.0000
Epoch 5/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 678730727424.0000 - val_loss: 757799780352.0000
Epoch 6/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 678656868352.0000 - val_loss: 757720547328.0000
Epoch 7/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 678582484992.0000 - val_loss: 757643083776.0000
Epoch 8/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 678508953600.0000 - val_loss: 7575643750

1155/1155 [==============================] - 3s 3ms/step - loss: 669575348224.0000 - val_loss: 748112642048.0000
Epoch 130/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 669500833792.0000 - val_loss: 748036620288.0000
Epoch 131/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 669427630080.0000 - val_loss: 747958894592.0000
Epoch 132/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 669354622976.0000 - val_loss: 747880054784.0000
Epoch 133/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 669281484800.0000 - val_loss: 747802525696.0000
Epoch 134/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 669207363584.0000 - val_loss: 747723423744.0000
Epoch 135/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 669134094336.0000 - val_loss: 747645566976.0000
Epoch 136/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 669060694016.0000 - val_loss: 747567579

1155/1155 [==============================] - 3s 3ms/step - loss: 660189609984.0000 - val_loss: 738179219456.0000
Epoch 258/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 660115947520.0000 - val_loss: 738101362688.0000
Epoch 259/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 660043857920.0000 - val_loss: 738028421120.0000
Epoch 260/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 659970457600.0000 - val_loss: 737949974528.0000
Epoch 261/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 659897057280.0000 - val_loss: 737872838656.0000
Epoch 262/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 659824508928.0000 - val_loss: 737795112960.0000
Epoch 263/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 659751043072.0000 - val_loss: 737716273152.0000
Epoch 264/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 659678494720.0000 - val_loss: 737639137

1155/1155 [==============================] - 3s 3ms/step - loss: 650870390784.0000 - val_loss: 728315527168.0000
Epoch 386/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 650798694400.0000 - val_loss: 728237342720.0000
Epoch 387/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 650726080512.0000 - val_loss: 728160337920.0000
Epoch 388/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 650653073408.0000 - val_loss: 728082808832.0000
Epoch 389/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 650580525056.0000 - val_loss: 728008294400.0000
Epoch 390/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 650508173312.0000 - val_loss: 727930699776.0000
Epoch 391/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 650435756032.0000 - val_loss: 727852646400.0000
Epoch 392/1000
1155/1155 [==============================] - 3s 3ms/step - loss: 650363142144.0000 - val_loss: 727775510

...

1171/1171 [==============================] - 2s 2ms/step - loss: 639472566272.0000 - val_loss: 710506905600.0000
Epoch 585/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 639399886848.0000 - val_loss: 710429638656.0000
Epoch 586/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 639327666176.0000 - val_loss: 710354075648.0000
Epoch 587/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 639254528000.0000 - val_loss: 710279626752.0000
Epoch 588/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 639182372864.0000 - val_loss: 710202359808.0000
Epoch 589/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 639110021120.0000 - val_loss: 710125617152.0000
Epoch 590/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 639037341696.0000 - val_loss: 710047105024.0000
Epoch 591/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 638964727808.0000 - val_loss: 709970821

1171/1171 [==============================] - 3s 2ms/step - loss: 630215933952.0000 - val_loss: 700736602112.0000
Epoch 713/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 630144434176.0000 - val_loss: 700661956608.0000
Epoch 714/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 630071230464.0000 - val_loss: 700585345024.0000
Epoch 715/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 629999599616.0000 - val_loss: 700508209152.0000
Epoch 716/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 629927444480.0000 - val_loss: 700433563648.0000
Epoch 717/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 629856010240.0000 - val_loss: 700357148672.0000
Epoch 718/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 629783527424.0000 - val_loss: 700281913344.0000
Epoch 719/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 629712093184.0000 - val_loss: 700205432

1171/1171 [==============================] - 3s 2ms/step - loss: 621026213888.0000 - val_loss: 691035242496.0000
Epoch 841/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 620955107328.0000 - val_loss: 690959351808.0000
Epoch 842/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 620883345408.0000 - val_loss: 690881822720.0000
Epoch 843/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 620811386880.0000 - val_loss: 690806063104.0000
Epoch 844/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 620740345856.0000 - val_loss: 690731417600.0000
Epoch 845/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 620668190720.0000 - val_loss: 690656444416.0000
Epoch 846/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 620597149696.0000 - val_loss: 690580815872.0000
Epoch 847/1000
1171/1171 [==============================] - 2s 2ms/step - loss: 620525322240.0000 - val_loss: 690505580

1171/1171 [==============================] - 3s 2ms/step - loss: 612025958400.0000 - val_loss: 681527672832.0000
Epoch 969/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 611956162560.0000 - val_loss: 681453158400.0000
Epoch 970/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 611886497792.0000 - val_loss: 681380347904.0000
Epoch 971/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 611817160704.0000 - val_loss: 681307668480.0000
Epoch 972/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 611747758080.0000 - val_loss: 681234202624.0000
Epoch 973/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 611677437952.0000 - val_loss: 681159753728.0000
Epoch 974/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 611608100864.0000 - val_loss: 681084649472.0000
Epoch 975/1000
1171/1171 [==============================] - 3s 2ms/step - loss: 611538501632.0000 - val_loss: 681014263

(827139.32421875, 0.947093165500861)

In [12]:
rmse_list

[885493.5234374999, 672669.3906249999, 865522.8476562501, 884871.5351562499]

In [13]:
mape_list

[0.9516994471800259,
 0.9368289128348756,
 0.9488051702830116,
 0.9510391317055306]